In [2]:
import digitalhub as dh
import os
PROJECT_NAME = "city-data"
project = dh.get_or_create_project(PROJECT_NAME)

## Open Data functions

In [3]:
new_folder ='src'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

In [7]:
func_download_buildings = project.new_function(name="download-buildings",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_buildings"},
                                               requirements= ["geopandas"])

In [8]:
run_download_buildings = func_download_buildings.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [9]:
func_download_addresses = project.new_function(name="download-addresses",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_addresses"},
                                               requirements= ["geopandas"])

In [10]:
run_download_addresses = func_download_addresses.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [11]:
func_download_walls = project.new_function(name="download-walls",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_walls"},
                                               requirements= ["geopandas"])

In [12]:
run_download_walls = func_download_walls.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [13]:
func_download_buidlings_volumes = project.new_function(name="download-buidlings-volumes",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_buidlings_volumes"},
                                               requirements= ["geopandas"])

In [14]:
run_download_buidlings_volumes = func_download_buidlings_volumes.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [15]:
func_download_buildings_symbols = project.new_function(name="download-buildings-symbols",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_buildings_symbols"},
                                               requirements= ["geopandas"])

In [16]:
run_download_buildings_symbols = func_download_buildings_symbols.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [17]:
func_download_terrain = project.new_function(name="download-terrain",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_terrain"},
                                               requirements= ["geopandas"])

In [18]:
run_download_terrain = func_download_terrain.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

In [19]:
func_download_areas = project.new_function(name="download-proximity-areas",
                                               kind="python",
                                               python_version="PYTHON3_10",
                                               source={"source": "src/download-open-data.py", "handler": "download_areas"},
                                               requirements= ["geopandas"])

In [20]:
run_download_areas = func_download_areas.run(action="job", parameters={'bucket': 'datalake'}, local_execution=False)

## Google Drive Functions

### Define and Build Functions: Download Traffic Spire

#### Create Secret Token

In order to read the files inside Google drive directory, it is required to create secret key,value pair inside to the project. Go to the 'Secret' section of project and create a new secret name 'token' with the value of Google drive token.

In [21]:
project.new_secret(name="token", secret_value="YOUR_TOKEN_VALUE")

In [27]:
func_download_lidar = project.new_function(
    name="download-lidar",
    kind="python",
    python_version="PYTHON3_10",
    source={"source": "src/download-data.py", "handler": "get_lidar"},
    requirements=['google-api-python-client', 'google_auth_oauthlib']
)

In [28]:
#!pip install google-api-python-client

In [29]:
run_lidar = func_download_lidar.run(action="job", parameters={'folder': 'LAS_CLASSIFICATO_TEST', 'bucket': 'datalake'}, local_execution=False)

In [30]:
func_download_dtm = project.new_function(
    name="download-dtm",
    kind="python",
    python_version="PYTHON3_10",
    source={"source": "src/download-data.py", "handler": "get_dtm"},
    requirements=['google-api-python-client', 'google_auth_oauthlib']
)

In [31]:
run_dtm = func_download_dtm.run(action="job", parameters={'folder': 'DTM_0.5_TEST', 'bucket': 'datalake'}, local_execution=False)

In [32]:
func_download_dsm = project.new_function(
    name="download-dsm",
    kind="python",
    python_version="PYTHON3_10",
    source={"source": "src/download-data.py", "handler": "get_dsm"},
    requirements=['google-api-python-client', 'google_auth_oauthlib']
)

In [33]:
run_dsm = func_download_dsm.run(action="job", parameters={'folder': 'DSM_0.5_TEST', 'bucket': 'datalake'}, local_execution=False)